## 读入数据

In [ ]:
import csv

f_1=csv.reader(open('../../../sources/taxi/train.csv', 'r', newline=''))
data=list(f_1)

## 抽取第一个坐标作为起始点

In [ ]:
pointList_2=[]
for i in range(2,len(data)-1):
    tmp_1=data[i][8].split('[')
    if (len(tmp_1)>2):
        tmp_2=tmp_1[2].split(']')
        tmp_3=tmp_2[0].split(',')
        tmp_3.append('1')

    pointList_2.append(tmp_3)

## 处理时间戳

In [ ]:
import datetime as dt
import pandas as pd
import pytz

source = pd.read_csv("../../../sources/taxi/train.csv", sep=",", low_memory=False)
#source = pd.read_csv('../../../sources/taxi/train.csv', sep=',', low_memory=False)
#source.head()
no_poly_source = source.loc[:, source.columns != 'POLYLINE']
#no_poly_source = source.loc[:]
#no_poly_source.head()
miss_false_no_poly_source = no_poly_source[no_poly_source.MISSING_DATA == False]

miss_false_no_poly_source.head()

In [ ]:
new_source = miss_false_no_poly_source.copy()
new_source['WEEK_DAY'] = new_source.TIMESTAMP.apply(lambda x: dt.datetime.utcfromtimestamp(x).weekday())
new_source['YEAR'] = new_source.TIMESTAMP.apply(lambda x: dt.datetime.utcfromtimestamp(x).year)
new_source['MONTH'] = new_source.TIMESTAMP.apply(lambda x: dt.datetime.utcfromtimestamp(x).month)
new_source['MONTH_DAY'] = new_source.TIMESTAMP.apply(lambda x: dt.datetime.utcfromtimestamp(x).day)
new_source['HOUR'] = new_source.TIMESTAMP.apply(lambda x: dt.datetime.utcfromtimestamp(x).hour)
new_source['DATE'] = new_source.TIMESTAMP.apply(lambda x: dt.datetime.utcfromtimestamp(x).date().isoformat())
new_source['DATE_DATE'] = new_source.TIMESTAMP.apply(lambda x: dt.datetime.utcfromtimestamp(x).date())
new_source.tail()

## 生产按小时分片的统计

In [ ]:
L_depart_time=[]
for i in range(24):
    L_depart_time.append([])

count=0 
    
for row in new_source.iterrows():
    hour=row[1][12]
    temp=pointList_2[count]
    count+=1
    L_depart_time[hour].append(temp)

## 写入csv

In [ ]:
for hour in range(24):
    filename="./TimeData/"+str(hour)+".csv"
    
    f_time_write=csv.writer(open(filename,'w', newline=''))
 
    L_title=['lng','lan','number']
    f_time_write.writerow(L_title)
    
    for item in L_depart_time[hour]:
        f_time_write.writerow(item)